# Implemetation of non-objective GIMME (GIMMES) for subnetwork generation

In [1]:
import cobra
import pandas as pd
import reframed
from reframed import load_cbmodel, from_cobrapy
from reframed.cobra.transcriptomics import GIMMES
from tools.designFunctions import set_medium, Media

### 1.Load expression data

In [2]:
species = 'pseudomonas_putida'
metadata_filepath = '/'.join(['data',species,'sample_table.csv'])
expression_filepath = '/'.join(['data',species,'log_tpm_norm.csv'])
metadata = pd.read_csv(metadata_filepath)
expression_data = pd.read_csv(expression_filepath).set_index('Geneid')
#Next step will consist on a function that will loc each experiment
#and according to the condition, it will choose the carbon_source
#and the media considered to eventually generate the context-model
#all this by querying the metadata

#By now just loc a predefined row that we have info about:
interest_condition = 'SRX4552613'
gene_exp = expression_data[interest_condition].to_dict()

### 2.Generate a media-specific model

In [5]:
#Load the model
model_name = 'iJN1463.xml'
model_filepath = '/'.join(['models', model_name])
cobra_model = cobra.io.read_sbml_model(model_filepath)
#Preprocess the model with metadata obtained from the previous step
media_name = 'm9'
media = Media(species, media_name)
carbon_source = {'EX_cit_e' : 8.0}
OD = 0.4
#od_coeff will be stored in designFunctions as a dict of species : coeffs
od_coeff = 0.6
observed_growth = od_coeff*OD
media_model = set_medium(cobra_model, media.get_predefined_media_definition(), carbon_source)
#Convert cobra model to reframed cbmodel
rf_media_model = from_cobrapy(media_model)
target_biomass = rf_media_model.biomass_reaction

### 3.Apply GIMMES over the configured model

In [7]:
gimmes_sol = GIMMES(rf_media_model, gene_exp, target_biomass, observed_growth)
gimmes_sol

KeyError: 'get_by_id'

In [ ]:
#it fails because i have developed the script as it was a cobra model.
#i need to change it to implement the same functionality but with the
#functions available in reframed

### 4.Identify Reaction Subnetworks